
start a bunch of front end services?


In [ ]:

const {spawn} = require("child_process");
const path = require('path')

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE

const COMMANDS = [
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'start a bunch of llm rpc services'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'start a bunch of discord services'],
  ['cloudflared', 'tunnel', 'run', '64d274ab-b0a4-4ef0-9636-e623dd02ac5c'],
  [path.join(HOMEPATH, 'stable-diffusion-webui', './webui.sh')],
  ['ollama', 'run', 'llama3.2-vision'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'express service', '--services=linkedin.ipynb'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'express service', '--services=git index.ipynb'],
  ['node', '--experimental-vm-modules', '-e', 'var result = require(\'./Core\').run()', '--', 'notebook lint watcher'],
]

const ENVIRONMENTS = [
  {
    SERVICE_NAME: 'llm-rpc'
  },
  {
    SERVICE_NAME: 'discord'
  },
  {
    SERVICE_NAME: 'cloud-flared'
  },
  {
    SERVICE_NAME: 'stable-webui'
  },
  {
    SERVICE_NAME: 'ollama-vision'
  },
  {
    // for emailing
    SERVICE_NAME: 'linked-out',
    ADMIN_USER: 'megamindbrian',
    ADMIN_PASS: JSON.parse(fs.readFileSync(path.join(HOMEPATH, '.credentials', 'google-secret.json'))),
    EXPRESS_PORT: 9191
  },
  {
    SERVICE_NAME: 'github-clone',
    EXPRESS_PORT: 9090
  },
  {
    SERVICE_NAME: 'lint-watcher',
  }
]

function launchChats(botIndex = 0) {

  if(parseInt(botIndex).toString() != botIndex.toString()) {
    return
  } else {
    botIndex = parseInt(botIndex)
  }

  if(typeof botIndex != 'number' && !botIndex) {
    return
  }

  for(let i = botIndex; i < ENVIRONMENTS.length; i++) {
    if(!ENVIRONMENTS[i] || !COMMANDS[i]) {
      continue
    }

    let child = spawn(COMMANDS[i][0], COMMANDS[i].slice(1), {
      env: Object.assign({}, process.env, ENVIRONMENTS[i] ? ENVIRONMENTS[i] : {}),
      stdio: ['pipe', 'pipe', 'pipe'],
      //detached: true,
      //shell: true,
      cwd: ENVIRONMENTS[i].CWD || path.dirname(__dirname)
    })

    child.stdout.on('data', (data) => console.log('[' + ENVIRONMENTS[i].SERVICE_NAME + ']', data.toString()))
    child.stderr.on('data', (data) => console.error('[' + ENVIRONMENTS[i].SERVICE_NAME + ']', data.toString()))

    if(botIndex) {
      break
    }
  }
}

module.exports = launchChats

